# Phase 3: Feature Engineering
## Step 3.1: Price Action Features
In this step, we derive features directly from the OHLCV (Open, High, Low, Close, Volume) data to capture market dynamics.

### Features to Implement:
1. **Candle Body:** Size and direction (`close - open`).
2. **Wicks:**
   - **Upper Wick:** `high - max(open, close)`
   - **Lower Wick:** `min(open, close) - low`
3. **Price Ratios:**
   - `close / open` (Daily return proxy)
   - `high / low` (Intraday volatility)
   - `close / high` (Strength near highs)
   - `close / low` (Strength near lows)
4. **Entry Price Context:** (`close` vs `entry_price` if available).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

# Add project root to path
project_root = Path('../').resolve()
sys.path.append(str(project_root))

from config import PROCESSED_DATA_DIR

# Set Plotting Style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

print("✅ Libraries Loaded")

In [ ]:
# Load Quality-Checked Data
data_path = PROCESSED_DATA_DIR / "step1_quality_checked.csv"

try:
    df = pd.read_csv(data_path)
    print(f"✅ Loaded Dataset from: {data_path}")
    print(f"Shape: {df.shape}")
except FileNotFoundError:
    print(f"❌ File not found: {data_path}. Please complete Phase 1 first.")

In [ ]:
# --- 1. Candle Properties ---
# Body Size (Absolute)
df['candle_body'] = df['close'] - df['open']

# Body Size (%)
df['candle_body_pct'] = (df['close'] - df['open']) / df['open']

# Upper Wick
df['upper_wick'] = df['high'] - df[['open', 'close']].max(axis=1)

# Lower Wick
df['lower_wick'] = df[['open', 'close']].min(axis=1) - df['low']

# Wick Ratio (Upper / Lower) - Handle division by zero
df['wick_ratio'] = df['upper_wick'] / (df['lower_wick'] + 1e-9)

# Total Range (High - Low)
df['candle_range'] = df['high'] - df['low']

# --- 2. Price Ratios ---
df['ratio_close_open'] = df['close'] / df['open']
df['ratio_high_low'] = df['high'] / df['low']
df['ratio_close_high'] = df['close'] / df['high']
df['ratio_close_low'] = df['close'] / df['low']

# --- 3. Entry Price Context (User Feature) ---
if 'entry_price' in df.columns:
    df['price_move_ratio'] = (df['close'] - df['entry_price']) / df['entry_price']
    print("✅ Added 'price_move_ratio' based on entry_price.")
else:
    print("ℹ️ 'entry_price' column not found, skipping price_move_ratio.")

print("✅ Price Action Features calculation complete.")

## Step 3.2: Volatility Features
Volatility is a key driver for trading signals. We will implement:
1. **Rolling Statistics:** Mean and Standard Deviation for different windows (7, 14, 50).
2. **Bollinger Bands:** 20-period moving average +/- 2 standard deviations.
3. **ATR (Average True Range):** Measure of market volatility.
4. **Percentage Range:** (High - Low) relative to Close.

In [ ]:
# Define Windows
windows = [7, 14, 50]

# 1. Rolling Statistics (Mean & Std)
for w in windows:
    df[f'rolling_mean_{w}'] = df['close'].rolling(window=w).mean()
    df[f'rolling_std_{w}'] = df['close'].rolling(window=w).std()
    
    # Normalized Deviation: (Close - Mean) / Std
    # This indicates how many standard deviations the price is away from the mean (Z-Score concept)
    df[f'z_score_{w}'] = (df['close'] - df[f'rolling_mean_{w}']) / (df[f'rolling_std_{w}'] + 1e-9)

print(f"✅ Rolling Mean/Std/Z-Score created for windows: {windows}")

# 2. Bollinger Bands (20, 2)
bb_window = 20
bb_std = 2
df['bb_middle'] = df['close'].rolling(window=bb_window).mean()
df['bb_upper'] = df['bb_middle'] + (df['close'].rolling(window=bb_window).std() * bb_std)
df['bb_lower'] = df['bb_middle'] - (df['close'].rolling(window=bb_window).std() * bb_std)

# BB Width: (Upper - Lower) / Middle -> Normalized measure of volatility
df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_middle']

# %B: (Close - Lower) / (Upper - Lower) -> Position of price within the bands
df['bb_pct_b'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'] + 1e-9)

# 3. Average True Range (ATR)
def calculate_atr(high, low, close, window=14):
    # True Range is max of: (High - Low), |High - PrevClose|, |Low - PrevClose|
    tr1 = high - low
    tr2 = (high - close.shift(1)).abs()
    tr3 = (low - close.shift(1)).abs()
    
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(window=window).mean()
    return atr

df['atr_14'] = calculate_atr(df['high'], df['low'], df['close'], window=14)
df['atr_pct'] = df['atr_14'] / df['close']

# --- Handle NaN Values (Clean Start) ---
# Drop rows where rolling windows haven't filled yet (using max window 50)
initial_len = len(df)
df.dropna(subset=['rolling_std_50', 'bb_upper', 'atr_14'], inplace=True)
print(f"✅ Dropped initial NaN rows. Rows removed: {initial_len - len(df)}")
print("✅ Volatility Features created.")

## Step 3.3: Momentum Features
Momentum indicators help identify the speed of price movement. We will implement:
1. **RSI Slope:** The rate of change of RSI (`RSI - RSI.shift(1)`).
2. **MACD:** Moving Average Convergence Divergence.
3. **ROC (Rate of Change):** Percentage change in price over `n` periods.
4. **Interaction Features:** Combining Momentum and Volatility (e.g., `RSI * BB_Width`).

In [ ]:
# Ensure RSI exists (from Phase 1). If not, calculate it.
def calculate_rsi(close, window=14):
    delta = close.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / (loss + 1e-9)
    return 100 - (100 / (1 + rs))

if 'RSI' not in df.columns:
    print("ℹ️ RSI not found, calculating it now...")
    df['RSI'] = calculate_rsi(df['close'])
else:
    print("✅ RSI already exists.")

# 1. RSI Slope (Momentum of Momentum)
df['rsi_slope'] = df['RSI'].diff()
df['rsi_slope_3'] = df['RSI'].diff(3) # 3-period slope

# 2. MACD (12, 26, 9)
exp1 = df['close'].ewm(span=12, adjust=False).mean()
exp2 = df['close'].ewm(span=26, adjust=False).mean()
df['macd'] = exp1 - exp2
df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
df['macd_hist'] = df['macd'] - df['macd_signal']

# 3. ROC (Rate of Change) - 7, 14 periods
for n in [7, 14]:
    df[f'roc_{n}'] = df['close'].pct_change(periods=n) * 100

# 4. Interaction Features
# Combining Volatility and Momentum
df['rsi_x_vol'] = df['RSI'] * df['bb_width']  # High RSI + High Volatility = Strong Move?
df['macd_x_vol'] = df['macd_hist'] * df['atr_pct'] # Strong MACD + High ATR

print("✅ Momentum Features (RSI Slope, MACD, ROC) created.")
print("✅ Interaction Features created.")

## Step 3.4: Feature Selection & Redundancy Check
To ensure efficient modeling, we must remove highly collinear features. Extremely high correlation (> 0.95) between features adds redundancy without adding information and can destabilize some models.

### Strategy:
1. Calculate the Feature-Feature Correlation Matrix.
2. Identify pairs with Correlation > 0.95.
3. From each pair, drop the feature with the **lower correlation to the target** (if available), or drop one arbitrarily.

In [ ]:
# Select only numeric columns for correlation
numeric_cols = df.select_dtypes(include=[np.number]).columns
exclude_cols = ['target_hit', 'stop_hit', 'target_type', 'hit_first', 'Unnamed: 0'] # Exclude non-features
feature_cols = [c for c in numeric_cols if c not in exclude_cols]

print(f"Initial Feature Count: {len(feature_cols)}")

# Calculate Correlation Matrix
corr_matrix = df[feature_cols].corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation > 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

print(f"⚠️ Highly Correlated Features to Drop (>0.95): {len(to_drop)}")

# Refined Drop Strategy: Keep the one better correlated with Target
if 'target_hit' in df.columns:
    final_drop_list = []
    target_corr = df[feature_cols].corrwith(df['target_hit']).abs()
    
    for col in to_drop:
        # Find the feature it is correlated with
        correlated_with = upper.index[upper[col] > 0.95].tolist()
        
        for other_col in correlated_with:
            # Compare target correlations
            score_current = target_corr.get(col, 0)
            score_other = target_corr.get(other_col, 0)
            
            # Logic: We want to DROP the one with LOWER score
            if score_current < score_other:
                final_drop_list.append(col)
            else:
                final_drop_list.append(other_col)
else:
    final_drop_list = to_drop

# --- Prevent Duplicates in Drop List (User Feedback) ---
final_drop_list = list(set(final_drop_list))

print(f"\n🔥 Final List of Features to Drop: {len(final_drop_list)}")
print(final_drop_list)

# Drop them
df_engineered = df.drop(columns=final_drop_list)
print(f"\n✅ Dropped redundant features. New Shape: {df_engineered.shape}")

In [ ]:
# --- Final Logging Summary (User Feedback) ---
print("\n📊 Feature counts by group:")
print(f"Price Action:  {len([c for c in df_engineered.columns if 'candle' in c or 'ratio_' in c or 'wick' in c])}")
print(f"Volatility:    {len([c for c in df_engineered.columns if 'rolling_' in c or 'bb_' in c or 'atr' in c or 'z_score' in c])}")
print(f"Momentum:      {len([c for c in df_engineered.columns if 'rsi' in c or 'macd' in c or 'roc_' in c])}")
print(f"Total Features: {len(df_engineered.columns)}")

# --- Save Final Engineered Data ---
output_path = PROCESSED_DATA_DIR / "step3_features_engineered.csv"
df_engineered.to_csv(output_path, index=False)

print(f"\n💾 Saved Final Engineered Dataset to: {output_path}")
print("✅ Phase 3 Refined & Complete!")